In [1]:
import os
import torch
import random

from miditok import REMI
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile
from pathlib import Path

os.environ['WANDB_DISABLED'] = 'true'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

seed = random.randint(1000, 10000)
tokenizer = REMI()

In [2]:
from torchtoolkit.data import create_subsets
from utils.midi_dataset import MIDIDataset

#tokens = tokenizer.load_tokens(Path('/home/nico/data/ai/models/midi/tokens.json'))
params = tokenizer.load_params(
    Path('/home/nico/data/ai/models/midi/token_params.json'))

midi_dataset = MIDIDataset(
    files_paths=list(
        Path('/home/nico/data/ai/models/midi/').glob('ozzy*.json')),
    min_seq_len=24,
    max_seq_len=128
)
subset_train, subset_valid = create_subsets(midi_dataset, [0.3])

len(subset_valid)

Loading data: /home/nico/data/ai/models/midi: 100%|██████████| 9/9 [00:00<00:00, 287.69it/s]


62

In [3]:
len(tokenizer)

221

In [4]:
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig

# Creates model
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=2048,
    n_embd=512,
    n_layer=8,
    n_head=8,
    n_inner=2048,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None']
)

model = GPT2LMHeadModel(config)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(221, 512)
    (wpe): Embedding(2048, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=221, bias=False)
)

In [5]:
from utils.midi_dataset import DataCollatorGen
from evaluate import load as load_metric
from torch import Tensor, argmax

metrics = {metric: load_metric(metric) for metric in ["accuracy"]}


def compute_metrics(eval_pred):
    """Computes metrics for pretraining.
    Must use proprocess_logits function that converts logits to predictions (argmax or sampling).

    :param eval_pred: EvalPrediction containing predictions and labels
    :return: metrics
    """
    predictions, labels = eval_pred
    not_pad_mask = labels != -100
    labels, predictions = labels[not_pad_mask], predictions[not_pad_mask]
    computed = metrics["accuracy"].compute(
        predictions=predictions.flatten(), references=labels.flatten())

    return computed


def preprocess_logits(logits: Tensor, _: Tensor) -> Tensor:
    """Preprocesses the logits before accumulating them during evaluation.
    This allows to significantly reduce the memory usage and make the training tractable.
    """
    pred_ids = argmax(logits, dim=-1)  # long dtype
    return pred_ids


training_config = TrainingArguments(
    "runs", False, True, True, False, "steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    eval_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=3.0,
    max_steps=1000,
    lr_scheduler_type="cosine_with_restarts",
    warmup_ratio=0.3,
    log_level="debug",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    metric_for_best_model='accuracy',
    greater_is_better=True,
    save_steps=100,
    save_total_limit=5,
    no_cuda=False,
    seed=seed,
    fp16=True,
    load_best_model_at_end=True,
    label_smoothing_factor=0.,
    optim="adamw_hf",
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    auto_find_batch_size=True
)

trainer = Trainer(
    model=model,
    args=training_config,
    data_collator=DataCollatorGen(tokenizer["PAD_None"]),
    train_dataset=subset_train,
    eval_dataset=subset_valid,
    compute_metrics=compute_metrics,
    callbacks=None,
    preprocess_logits_for_metrics=preprocess_logits,
)

# Training
train_result = trainer.train()
trainer.save_model()  # Saves the tokenizer too
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
/home/nico/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 145
  Num Epochs = 28
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 1,000
  Number of trainable parameters = 26,381,824


  0%|          | 0/1000 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 5.2089, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.55}
{'loss': 3.9942, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.1}
{'loss': 3.2065, 'learning_rate': 2e-05, 'epoch': 1.64}
{'loss': 2.7077, 'learning_rate': 2.6666666666666667e-05, 'epoch': 2.19}


***** Running Evaluation *****
  Num examples = 62
  Batch size = 2


...

{'eval_loss': 1.173716425895691, 'eval_accuracy': 0.00847672064777328, 'eval_runtime': 1.4361, 'eval_samples_per_second': 43.174, 'eval_steps_per_second': 21.587, 'epoch': 27.4}


Model weights saved in runs/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [runs/checkpoint-500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from runs/checkpoint-700 (score: 0.010880566801619434).
Saving model checkpoint to runs
Configuration saved in runs/config.json
Configuration saved in runs/generation_config.json


{'train_runtime': 304.041, 'train_samples_per_second': 13.156, 'train_steps_per_second': 3.289, 'train_loss': 1.0806627798080444, 'epoch': 27.4}


Model weights saved in runs/pytorch_model.bin


***** train metrics *****
  epoch                    =       27.4
  train_loss               =     1.0807
  train_runtime            = 0:05:04.04
  train_samples_per_second =     13.156
  train_steps_per_second   =      3.289


In [9]:
import json

from typing import Any, Dict, List
from torch import LongTensor, flip, cat, full
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from copy import deepcopy
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained('./runs/')
model = model.to(device)

def collate_gen_left(batch: List[Dict[str, LongTensor]]) -> LongTensor:
    # Here the sequences are padded to the left, so that the last token along the time dimension
    # is always the last token of each seq, allowing to efficiently generate by batch
    bos_shape = (1,)
    batch = [flip(cat([full(bos_shape, tokenizer["BOS_None"]),
                  seq["input_ids"]], dim=0), dims=(0,)) for seq in batch]
    batch = pad_sequence(batch, batch_first=True,
                         padding_value=tokenizer["PAD_None"])  # (N,T) or (N,T,Z)
    batch = flip(batch, dims=(1,)).long()
    return batch  # (N,T)


generation_config = GenerationConfig(
    min_new_tokens=512,
    max_new_tokens=1024,
    repetition_penalty=1.5,
    num_beams=24,        # no beam search
    early_stopping=True,
    no_repeat_ngram_size=5,
    num_return_sequences=16,
    length_penalty=0.5,
    num_beam_groups=4,
    diversity_penalty=2.0,
    do_sample=False,     # but sample instead
    temperature=0.75,
    top_k=35,
    top_p=0.35,
    epsilon_cutoff=3e-4,
    eta_cutoff=1e-3,
    pad_token_id=config.padding_token_id,
)

(gen_results_path := Path('gen_res')).mkdir(parents=True, exist_ok=True)

def rec_gen(tokens):
    global model
    global generation_config

    res = model.generate(torch.LongTensor([tokens]).to(model.device),
                         generation_config=generation_config)
    
    out = res[0].cpu().numpy().tolist()
    new_tokens = out[len(tokens)-1:]

    print(f'Generated {len(new_tokens)} new tokens.')

    return new_tokens


max_iter = 2
iter_count = 0
init_size = 256

with open('/home/nico/data/ai/models/midi/ozzy_osbourne-facing_hell.json') as tokens_file:
    ids = json.load(tokens_file)['ids']
    tokens = ids[0][:init_size] # 1 channel only

    while iter_count < max_iter:
        block_size = init_size if iter_count == 0 else int(init_size / 128)
        tokens += rec_gen(tokens[-block_size:])

        iter_count += 1

loading configuration file ./runs/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 512,
  "n_head": 8,
  "n_inner": 2048,
  "n_layer": 8,
  "n_positions": 2048,
  "padding_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.29.0.dev0",
  "use_cache": true,
  "vocab_size": 221
}

loading weights file ./runs/pytorch_model.bin
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "tran

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:74                                                                                   │
│                                                                                                  │
│   71 │                                                                                           │
│   72 │   while iter_count < max_iter:                                                            │
│   73 │   │   block_size = init_size if iter_count == 0 else int(init_size / 128)                 │
│ ❱ 74 │   │   tokens += rec_gen(tokens[-block_size:])                                             │
│   75 │   │                                                                                       │
│   76 │   │   iter_count += 1                                                                     │
│   77                                                                                             │
│                                                                                                  │
│ in rec_gen:53                                                                                    │
│                                                                                                  │
│   50 │   global model                                                                            │
│   51 │   global generation_config                                                                │
│   52 │                                                                                           │
│ ❱ 53 │   res = model.generate(torch.LongTensor([tokens]).to(model.device),                       │
│   54 │   │   │   │   │   │    generation_config=generation_config)                               │
│   55 │                                                                                           │
│   56 │   out = res[0].cpu().numpy().tolist()                                                     │
│                                                                                                  │
│ /home/nico/anaconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py:115 in              │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /home/nico/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1601 in       │
│ generate                                                                                         │
│                                                                                                  │
│   1598 │   │                                                                                     │
│   1599 │   │   elif is_group_beam_gen_mode:                                                      │
│   1600 │   │   │   if generation_config.num_return_sequences > generation_config.num_beams:      │
│ ❱ 1601 │   │   │   │   raise ValueError("`num_return_sequences` has to be smaller or equal to `  │
│   1602 │   │   │                                           

In [7]:

print('Generating the midi...')

midi = tokenizer.tokens_to_midi(torch.LongTensor([tokens]).cpu(), time_division=384)
# midi.instruments[0].name = f'Continuation of original sample ({len(generated)} tokens)'
# midi.instruments[1].name = f'Original sample ({len(prompt)} tokens)'
# midi.instruments[2].name = f'Original sample and continuation'
midi.dump(gen_results_path / 'full.mid')
# tokenizer.save_tokens(tokens, gen_results_path / f'{count}.json')

type(midi)

Generating the midi...


miditoolkit.midi.parser.MidiFile